# Replace BG Dataset

## 1. Download and configure dataset
- Download `REPLACE-BG-Dataset.zip` from this [link](https://public.jaeb.org/datasets/diabetes);
- Create a folder in `fedbiomed/data/` named `replace-bg`;
- Unzip `REPLACE-BG-Dataset.zip` and move it in `{FEDBIOMED_DIR}/data/replace-bg`;
- Run `python ${FEDBIOMED_DIR}/docs/tutorials/sec-agg/client-selection/preprocessing.py`

## 2. Add 202 nodes and start --num_nodes nodes (up to 202)
- Open a terminal and run `python ${FEDBIOMED_DIR}/docs/tutorials/client-selection/add_nodes.py`
- run `python ${FEDBIOMED_DIR}/docs/tutorials/sec-agg/client-selection/start_nodes.py --num_nodes 10`

In [9]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets,transforms
import datetime
import numpy as np
from torch.utils.data import Dataset
import os
import pandas as pd
FEDBIOMED_DIR = os.environ.get('FEDBIOMED_DIR')

DATA_DIR = f"{FEDBIOMED_DIR}/data/replace-bg/raw/patients"


# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    
    def __init__(self):
        super().__init__()
        self.input_length = 12 # it mean past 3 hours
        self.pred_length = 4 # it mean next hour
    
    # Defines and return model 
    def init_model(self, model_args):
        return self.CNN_LSTM(model_args = model_args)
    
    # Defines and return optimizer
    def init_optimizer(self, optimizer_args):
        return torch.optim.SGD(self.model().parameters(), lr = optimizer_args["lr"])
    
    # Declares and return dependencies
    def init_dependencies(self):
        deps = ["import pandas as pd", "from torch.utils.data import Dataset", "import numpy as np", "import datetime"]
        return deps
    

    class CNN_LSTM(nn.Module):
        def __init__(self, model_args):
            super().__init__()
            # Channels for prediction (assuming one-dimensional time-series data)
            predict_channels = [0]
            self.predict_channels = predict_channels

            # Define the convolutional layers
            self.conv_layers = nn.Sequential(
                nn.Conv1d(in_channels=3, out_channels=32, kernel_size=7, padding=3),
                nn.ReLU(),
                nn.Conv1d(in_channels=32, out_channels=64, kernel_size=7, padding=3),
                nn.ReLU(),
                nn.MaxPool1d(kernel_size=2),  # MaxPool reduces dimensionality
                nn.Conv1d(in_channels=64, out_channels=64, kernel_size=5, padding=2),  # Added padding to maintain dimensions
                nn.ReLU(),
                nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding=2),  # Added padding to maintain dimensions
                nn.ReLU(),
                nn.MaxPool1d(kernel_size=2),  # MaxPool reduces dimensionality
            )

            # Define the LSTM layers
            self.lstm = nn.LSTM(input_size=128, hidden_size=100, num_layers=2, batch_first=True, dropout=0.2)

            # Define the fully connected layers
            self.fc_layers = nn.Sequential(
                nn.Linear(100, 64),
                nn.Tanh(),
                nn.Linear(64, 6),
                nn.Tanh(),
                nn.Linear(6, len(predict_channels))
            )

        def _forward(self, x):
            """
            Perform forward pass through the CNN and LSTM layers.

            Args:
                x: Input tensor with shape (batch_size, input_length, num_features)

            Returns:
                Output tensor after passing through the CNN and LSTM layers
            """
            x = x.permute(0, 2, 1)  # Change shape to (batch_size, num_features, input_length) for Conv1d
            x = self.conv_layers(x)
            x = x.permute(0, 2, 1)  # Change back to (batch_size, input_length, num_features) for LSTM
            x, _ = self.lstm(x)
            x = self.fc_layers(x[:, -1, :])  # Use the output of the last LSTM cell
            return x

        def forward(self, whole_example, input_length):
            """
            Perform sequential prediction using teacher forcing.

            Args:
                whole_example: Input tensor with shape (batch_size, total_length, num_features)
                input_length: Length of the sequence to use as input initially

            Returns:
                Tensor with the same shape as whole_example, where the values from input_length
                onward have been replaced with model predictions
            """
            whole_example_clone = whole_example.clone().detach()  # Create a detached clone of the input tensor
            total_length = whole_example_clone.shape[1]  # Get the total length of the sequence
            assert input_length < total_length, "input_length should be less than total_length"

            # Sequentially predict and replace the values in the cloned tensor
            # This process uses teacher forcing: it feeds the true previous (up to :input_length) values as input
            # And from [input_length:total_length] it feeds the prediction
            while input_length < total_length:
                # Use the sequence up to the current input_length as input
                x = whole_example[:, :input_length, :]
                # Perform forward pass to get the prediction for the next time step
                y_hat = self._forward(x)
                # Replace the value at the current input_length position with the prediction
                whole_example_clone[:, input_length, self.predict_channels] = y_hat[:, self.predict_channels]
                # Increment input_length to move to the next position
                input_length += 1

            return whole_example_clone

    class ReplaceBGDataset(Dataset):
        """The Replace-BG dataset for Torch training."""
    
        def __init__(self, raw_df, example_len, unimodal=False):
            """
            Args
                raw_df: dataframe
                example_len: int
                unimodal: bool
                    If True, data contains glucose only
            """
            raw_df.replace(to_replace=-1, value=np.nan, inplace=True)
            self.example_len = example_len
            self.unimodal = unimodal
            self.data, self.times = self._initial(raw_df)  # (len, n_features)
            self.example_indices = self._example_indices(self.times)
            self.external_mean = [160.87544032,   0.21893523,   1.49783614]
            self.external_std = [63.60143682,  1.14457581,  8.92042825]
            self._standardise(self.external_mean, self.external_std)
            print("Dataset loaded, total examples: {}.".format(len(self)))
    
            # post check
            # for i in range(len(self)):
            #     if torch.isnan(self[i]).any():
            #         raise ValueError("NaN detected in dataset!")
    
        @staticmethod
        def str2dt(s):
            return datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S")
    
        def _initial(self, raw_df):
            times = [self.str2dt(s) for s in raw_df["time"]]
            glucose = raw_df["GlucoseValue"].to_numpy(dtype=np.float32)
            bolus = raw_df["Normal"].to_numpy(dtype=np.float32)
            carbs = raw_df["CarbInput"].to_numpy(dtype=np.float32)
    
            bolus[np.isnan(bolus)] = 0.0
            carbs[np.isnan(carbs)] = 0.0
    
            return (
                np.array(
                    [
                        glucose,
                        bolus,
                        carbs,
                    ],
                    dtype=np.float32,
                ).T,
                times,
            )
    
        def _example_indices(self, times):
            """Extract every possible example from the dataset, st. all data entry in this example is not missing.
    
            Returns:
                [(start_row, end_row)]
                    Starting and ending indices for each possible example from this dataframe.
            """
            res = []
            total_len = self.data.shape[0]
    
            def look_ahead(start):
                end = start
                res = []
                while end < total_len:
                    if np.any(np.isnan(self.data[end, :])):
                        break
                    if end - start + 1 >= self.example_len:
                        # check that between start and end, there is the difference of self.example_len * 15 minutes
                        gap_min = self.example_len * 15
                        if (times[end] - times[end - self.example_len + 1]) <= datetime.timedelta(minutes=gap_min):
                            res.append((end - self.example_len + 1, end))
                    end += 1
                return res, end
    
            i = 0
            while i < total_len:
                if not np.any(np.isnan(self.data[i, :])):
                    temp_res, temp_end = look_ahead(i)
                    res += temp_res
                    i = temp_end + 1
                else:
                    i += 1
            return res
    
        def _standardise(self, external_mean=None, external_std=None):
            if external_mean is None and external_std is None:
                mean = []
                std = []
                for i in range(self.data.shape[1]):
                    mean.append(np.nanmean(self.data[:, i]))
                    std.append(np.nanstd(self.data[:, i]))
            else:
                mean = external_mean
                std = external_std
            self.mean = mean
            self.std = std
            print("Standardising with mean: {} and std: {}.".format(mean, std))
            for i in range(self.data.shape[1]):
                self.data[:, i] = (self.data[:, i] - mean[i]) / std[i]
    
        def __len__(self):
            return len(self.example_indices)
    
        def __getitem__(self, idx):
            """
            Args:
                idx: int
            Returns:
                (example_len, channels)
            """
            start_row, end_row = self.example_indices[idx]
            res = torch.from_numpy(self.data[start_row : end_row + 1, :])
            # print(f"start_row: {self.times[start_row]}, end_row: {self.times[end_row +1]}")
            return res,0  #Dummy variable to be compatible with TorchTrainingPlan

    def training_data(self):
        dataset1 = self.ReplaceBGDataset(raw_df=pd.read_csv(self.dataset_path), example_len=16)
        train_kwargs = { 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        target = data[:, -self.pred_length :, 0]
        output = self.model().forward(data, self.input_length)[:, -self.pred_length :, 0] 
        loss = torch.nn.functional.l1_loss(output, target)
        return loss


2024-05-26 20:10:07,714 fedbiomed DEBUG - Node: NODE_3e1e57c8-1364-41cc-8854-15c82e9d5b93 polling for the tasks

2024-05-26 20:10:15,949 fedbiomed DEBUG - Node: NODE_b14a8bc6-ff16-4c92-8772-4a755ad2855f polling for the tasks

2024-05-26 20:10:18,324 fedbiomed DEBUG - Node: NODE_61dbf902-17f9-4bc9-b9f2-d46bfdca0ec5 polling for the tasks

In [3]:
model_args = {}

training_args = {
    'loader_args': { 'batch_size': 64, }, 
    'optimizer_args': {
        "lr" : 1e-3
    },
    # 'use_gpu': True,  # automatically falls back to cpu on nodes that don't support gpu
    'num_updates': 20, 
    'dry_run': False,  
    'random_seed':42, 
    'log_interval': 20,
}

In [4]:
from fedbiomed.researcher.strategies import DefaultStrategy
import random
class UniformSelectStrategy(DefaultStrategy):
    
    def __init__(self, sampling_fraction = 1.0):
        super().__init__()
        """
        Initialize the class with a sampling fraction.

        Args:
            sampling_fraction (float): Fraction of nodes to sample each round. Must be between 0 and 1.
        """
        self.sampling_fraction = sampling_fraction

    def sample_nodes(self, from_nodes, round_i):
        """
        Samples and selects nodes on which to train the local model. This strategy considers a subsample of existing nodes.

        Args:
            from_nodes: List of node ids which may be sampled.
            round_i: Number of the round.

        Returns:
            List of node ids considered for training during this round `round_i`.
        """
        random.seed(42 + round_i)  # Using round_i to ensure different seeds for different rounds
        num_nodes_to_sample = int(len(from_nodes) * self.sampling_fraction)
        sampled_nodes = random.sample(from_nodes, num_nodes_to_sample)

        self._sampling_node_history[round_i] = sampled_nodes

        return sampled_nodes

In [5]:
from fedbiomed.researcher.federated_workflows import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage
from fedbiomed.researcher.secagg import SecureAggregation


tags =  ['replace-bg']
rounds = 150

exp = Experiment(tags=tags,
                 model_args=model_args,
                 training_plan_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 # secagg=SecureAggregation(active=True, scheme='flamingo'), # or custom SecureAggregation(active=<bool>, clipping_range=<int>)
                 node_selection_strategy=UniformSelectStrategy(0.09))
exp.set_retain_full_history(True)


2024-05-26 10:57:53,569 fedbiomed INFO - Starting researcher service...

2024-05-26 10:57:53,572 fedbiomed INFO - Waiting 3s for nodes to connect...

2024-05-26 10:57:53,781 fedbiomed DEBUG - Node: NODE_73dfa61c-7e43-4c3d-b37c-8c6fd7fc731e polling for the tasks

2024-05-26 10:57:53,785 fedbiomed DEBUG - Node: NODE_19ba773a-cfe2-4a33-b93b-99e4ae6ba6b7 polling for the tasks

2024-05-26 10:57:53,786 fedbiomed DEBUG - Node: NODE_176ee04a-b644-4f0d-828a-5635e105bddc polling for the tasks

...

2024-05-26 10:57:58,647 fedbiomed INFO - Node selected for training -> NODE_b1662e51-f4c1-4cb2-a291-d15366a2d5c6

2024-05-26 10:57:58,647 fedbiomed INFO - Node selected for training -> NODE_2ed77e44-172c-40f5-9070-e50ba49c749c

2024-05-26 10:57:58,659 fedbiomed DEBUG - Model file has been saved: /workspaces/Projects/fedbiomed/var/experiments/Experiment_0055/model_3aa7f279-864e-43f4-b828-122e2e7ae1b0.py

Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.


True

In [6]:
exp.run()

2024-05-26 10:58:06,490 fedbiomed INFO - Sampled nodes in round 0 ['NODE_5fb9ae1b-15a3-4e10-bb71-f8ee74e5f5bf', 'NODE_41da6abb-cca3-404d-8304-b6a03149c61c', 'NODE_cefc6129-266e-4ae5-a4b4-d453fae95a33', 'NODE_e9b0856f-5bdb-4d13-8507-f22b2415d4d6', 'NODE_81a678b4-cb46-4d2f-8908-12d41128b9ab', 'NODE_325510de-b109-4a1c-93c3-579e65cf5895', 'NODE_ef68009c-2780-4ceb-8b0f-83763aaa4766', 'NODE_1a806e6c-de81-4e80-bc5a-7bf2a9396298', 'NODE_6a2cbcc2-abd6-43c2-89f5-d651cca8c790', 'NODE_077a81e2-7093-46d6-a1d2-bc75ae8d8cef', 'NODE_f390aef6-11ae-4924-a71f-79b7bb7018d4', 'NODE_06a86905-61b8-4aca-a0ab-2d885adee370']

2024-05-26 10:58:06,494 fedbiomed INFO - Sending request 
					 To: NODE_5fb9ae1b-15a3-4e10-bb71-f8ee74e5f5bf 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-05-26 10:58:06,495 fedbiomed INFO - Sending request 
					 To: NODE_41da6abb-cca3-404d-8304-b6a03149c61c 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-05-26 10:58:06,496 fedbiomed INFO - Sending request 
					 To: NODE_cefc6129-266e-4ae5-a4b4-d453fae95a33 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-05-26 10:58:06,496 fedbiomed INFO - Sending request 
					 To: NODE_e9b0856f-5bdb-4d13-8507-f22b2415d4d6 
					 Request: : TRAIN
 -----------------------------------------------------------------

...

2024-05-26 20:09:17,707 fedbiomed DEBUG - Node: NODE_90597543-fe37-4fa5-9fb8-ad0fb35c66f3 polling for the tasks

2024-05-26 20:09:23,612 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_360e1c17-9433-4903-be13-7a85a76bc151 
					 Round 108 | Iteration: 20/20 (100%) | Samples: 1280/1280
 					 Loss: 0.769884 
					 ---------


--------------------
Fed-BioMed researcher stopped due to unknown error:
local variable 'replies' referenced before assignment
More details in the backtrace extract below
--------------------
Traceback (most recent call last):
  File "/workspaces/Projects/fedbiomed/fedbiomed/researcher/federated_workflows/_federated_workflow.py", line 49, in payload
    ret = function(*args, **kwargs)
  File "/workspaces/Projects/fedbiomed/fedbiomed/researcher/federated_workflows/_experiment.py", line 774, in run_once
    training_replies, aux_vars = job.execute()
  File "/workspaces/Projects/fedbiomed/fedbiomed/researcher/federated_workflows/jobs/_training_job.py", line 190, in execute
    training_replies = self._get_training_results(replies=replies,
UnboundLocalError: local variable 'replies' referenced before assignment
--------------------


2024-05-26 20:09:41,335 fedbiomed CRITICAL - Fed-BioMed stopped due to unknown error:
local variable 'replies' referenced before assignment

FedbiomedSilentTerminationError: 

In [7]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
for rnd in range(rounds):
    round_data = exp.training_replies()[rnd]
    for r in round_data.values():
        print("\t- {id} :\
        \n\t\trtime_training={rtraining:.2f} seconds\
        \n\t\tptime_training={ptraining:.2f} seconds\
        \n\t\trtime_total={rtotal:.2f} seconds".format(id = r['node_id'],
            rtraining = r['timing']['rtime_training'],
            ptraining = r['timing']['ptime_training'],
            rtotal = r['timing']['rtime_total'],))
    print('\n')


List the training rounds :  dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106])

List the nodes for the last training round and their timings : 
	- NODE_5fb9ae1b-15a3-4e10-bb71-f8ee74e5f5bf :        
		rtime_training=319.13 seconds        
		ptime_training=202.56 seconds        
		rtime_total=321.59 seconds
	- NODE_41da6abb-cca3-404d-8304-b6a03149c61c :        
		rtime_training=316.31 seconds        
		ptime_training=201.97 seconds        
		rtime_total=321.59 seconds
	- NODE_cefc6129-266e-4ae5-a4b4-d453fae95a33 :        
		rtime_training=314.29 seconds        
		ptime_training=200.66 seconds        
		rtim

KeyError: 107

In [14]:
patients = [f for f in os.listdir(DATA_DIR) if f.endswith('.csv')]
patients.sort()
patients = [int(p.replace(".csv", "")) for p in patients]

patients_testing = patients[180:]
test_dataset = torch.utils.data.ConcatDataset(
        [
            MyTrainingPlan.ReplaceBGDataset(
                raw_df=pd.read_csv(os.path.join(DATA_DIR, "{}.csv".format(p))),
            example_len=16)
            for p in patients_testing
        ])

Standardising with mean: [160.87544032, 0.21893523, 1.49783614] and std: [63.60143682, 1.14457581, 8.92042825].
Dataset loaded, total examples: 18054.
Standardising with mean: [160.87544032, 0.21893523, 1.49783614] and std: [63.60143682, 1.14457581, 8.92042825].
Dataset loaded, total examples: 2586.
Standardising with mean: [160.87544032, 0.21893523, 1.49783614] and std: [63.60143682, 1.14457581, 8.92042825].
Dataset loaded, total examples: 20509.
Standardising with mean: [160.87544032, 0.21893523, 1.49783614] and std: [63.60143682, 1.14457581, 8.92042825].
Dataset loaded, total examples: 19895.
Standardising with mean: [160.87544032, 0.21893523, 1.49783614] and std: [63.60143682, 1.14457581, 8.92042825].
Dataset loaded, total examples: 16179.
Standardising with mean: [160.87544032, 0.21893523, 1.49783614] and std: [63.60143682, 1.14457581, 8.92042825].
Dataset loaded, total examples: 15405.
Standardising with mean: [160.87544032, 0.21893523, 1.49783614] and std: [63.60143682, 1.144575

In [16]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader


def testing_rmse(model, data_loader):
    model.eval()
    total_squared_error = 0.0
    total_samples = 0
    input_length = 12
    pred_length = 4

    device = "cpu"

    with torch.no_grad():
        for idx, (data,target) in enumerate(data_loader):
            target = data[:, -pred_length :, 0]
            target = target * 63.60143682 + 160.87544032
            data, target = data.to(device), target.to(device)
            output = model(data, input_length)[:, -pred_length :, 0]
            output = output * 63.60143682 + 160.87544032

            # Assuming the model output and target are continuous values
            mse = F.mse_loss(output, target, reduction='sum').item()
            total_squared_error += mse
            total_samples += len(data)

            # Only uses 10% of the dataset for faster results
            if idx >= len(data_loader) / 10:
                break
    
    # Calculate the Root Mean Squared Error
    rmse = torch.sqrt(torch.tensor(total_squared_error / total_samples))

    return rmse.item()

batch_size = 32  # Adjust as needed
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
for r in range(rounds):
    fed_model = exp.training_plan().model()
    fed_model.load_state_dict(exp.aggregated_params()[r]['params'])
    print(testing_rmse(fed_model,test_loader))

143.43421936035156
143.565185546875
143.51353454589844
143.60012817382812
143.7473907470703
143.89306640625
143.95703125
143.8656005859375
143.8738555908203
144.01394653320312
143.94305419921875
143.94717407226562
144.00003051757812
